# ML App to Create a Custom Newsfeed

## Introduction:

Reading is an excellent way to fill in any knowledge gap. It is even better if there is a way to enchance this experience whereby to quickly get a customised newsfeed tailored to the reader's interests. This project/app aims to help frequent readers such as myself to find even more to read or rather, to satisfy curiousities on certain topics. It can be seen as beneficial to have what you want to read without going through the hassle of digging for it. 

Hopefully by the end of this project, a system can be built that is able to understand a tailored taste in news and be able to send them directly to the user.

## Breakdown of the Project:
- Making a supervised training set with the Pocket App.
- Utilising the Pocket API to obtain stories.
- Making use of the Embedly API to extract story bodies.
- Basics of NLP (Natural Language Processing)
- Classifiers such as SVM (Support Vector Machines)
- IFTTT will be integrated with RSS feeds and Google Sheets.
- Setup a system for a daily personal newsletters

## Installing Pocket Chrome Extension for this project:

The Pocket app/extension allows for saving personalised stories to be read later on. The stories can be linked to tags, and these tags can be different to every user. From this, the idea is to use to to tag 'y' for interesting topics while 'n' for non-interesting topics. Usage of the Pocket app/extension to help create a __supervised training__ set for the model.

Setup and example of usage:
1. Open Chrome, and add Pocket app in the Extensions.
2. Open any link of interest.
3. Click on the Pocket extension icon, once red, it means the article or link have been saved.

To begin constructing the supervised dataset:
1. As you go through your day, read your articles.
2. Tag 'y' for interesting ones, and tag 'n' for non-interesting ones.
3. Note: it is better to have lots (100s) of articles to make the model better.

Retrive the save stories:
1. Go to: https://getpocket.com/developer/apps/new
2. Click on 'Create a New App' in the top left corner of the webpage.
3. Input the details and get the API KEY.
4. Be sure to click on all the Permissions (allows for adding, changing and retrieving)
5. Once done, click to submit.
6. Click on 'My Apps' in the top left corner fo the webpage and you will see the custom applications.

After these, move on to the next stage which is the code below.

## 1 Import the required Libraries:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os

## 2 Setup Authorisations:

This section requires the Pocket API key (consumer key) and a redirect URL. Note that the redirect URL can be anything here. For example, it can be a twitter account  or otherwise. For more information on Redirect URL: https://www.hostgator.com/blog/ways-redirect-website-url/.

In this case, a twitter account was used.

In [2]:
pd.set_option('display.max_colwidth', 200)

# Get Pocket API Key:
def get_API_KEY(APIKEY_fileName):
    currentDirectory = os.getcwd()
    path_to_API_KEY = currentDirectory + '/' + APIKEY_fileName
    API_KEY_file = open(path_to_API_KEY)
    
    API_KEY = []
    for i in API_KEY_file:
        API_KEY.append(i)
    
    return str(API_KEY[0])

In [3]:
pocket_API_key = get_API_KEY('Pocket API Key.txt')
print(pocket_API_key)

89714-63913284a8c8d0a2e346b63d


In [4]:
# Get Redirect URL from text file:
def get_REDIRECT_URL(RURL_fileName):
    currentDirectory = os.getcwd()
    RURL_path = currentDirectory + '/' + RURL_fileName
    URL_file = open(RURL_path)
    
    RedirectURL = []
    for i in URL_file:
        RedirectURL.append(i)
    
    return str(RedirectURL[0])

In [5]:
Redirect_URL_key = get_REDIRECT_URL('TwitterAccount.txt')
print(Redirect_URL_key)

https://www.twitter.com/YSLee65720613


In [6]:
# Authentication Parameters:
auth_params = {'consumer_key': pocket_API_key, 'redirect_uri': Redirect_URL_key}

token = requests.post('https://getpocket.com/v3/oauth/request', data = auth_params)

print('Access code is: ')
print(token.text)

Access code is: 
code=8091ee4b-9542-1e6c-bcdb-41dd1b


#### Next, is to copy the following output link and AUTHORISE it:

In [7]:
# Generate the new output link:
auth_Link = 'https://getpocket.com/auth/authorize?request_token=' + token.text.strip('code=') + '&redirect_uri=https%3A//' + Redirect_URL_key.strip('https://')
print(auth_Link)

https://getpocket.com/auth/authorize?request_token=8091ee4b-9542-1e6c-bcdb-41dd1b&redirect_uri=https%3A//www.twitter.com/YSLee65720613


#### This is then followed by Parsing out the access code:

In [11]:
# Parsing out the access vode from token.text string;
access_code = token.text.split('=')[1]

user_params = {'consumer_key': pocket_API_key, 'code': access_code}

user = requests.post('https://getpocket.com/v3/oauth/authorize', data = user_params)

print('User code is: ')
print(user.text)

User code is: 
access_token=c3427703-c4f7-a7cc-4470-502f4a&username=ylee9106%40gmail.com


## 3 Retrieving the Saved Online Articles: 

Utilising the output 'User Code' above, the taged stories can be retrieved. 

### 3.1 Begin with 'n' tags: